In [1]:
import os
import zipfile
import requests
import json
from bioservices import BioModels

In [2]:
s = BioModels()

INFO    [bioservices.BioModels:363]:  Initialising BioModels service (REST)


In [3]:
def get_all_models(query, page_size=10):
    """
    Fonction pour récupérer tous les modèles correspondant à la requête avec pagination.
    """
    models = []
    offset = 0

    while True:
        try:
            search_results = s.search(query, offset=offset)
            
            if 'models' not in search_results or not search_results['models']:
                break
            
            models.extend(search_results['models'])
            offset += page_size
            print(f"Page {offset // page_size} téléchargée, {len(search_results['models'])} modèles récupérés.")
        
        except Exception as e:
            print(f"Erreur lors de la récupération des modèles : {e}")
            break
    
    return models

In [10]:
def download_model_file(model_id, directory):
    """
    Télécharge le fichier ZIP du modèle BioModels, extrait uniquement le premier fichier .xml,
    et supprime le ZIP ensuite.
    """
    try:
        # Créer le chemin du ZIP
        zip_filename = f"{model_id}.zip"
        zip_path = os.path.join(directory, zip_filename)

        # Télécharger le ZIP
        s.get_model_download(model_id, output_filename=zip_path)

        # Extraire uniquement le premier fichier .xml trouvé
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            xml_files = [f for f in zip_ref.namelist() if f.endswith('.xml') and not f.lower().endswith('_urn.xml') and not f.lower().endswith('manifest.xml')]
            if not xml_files:
                print(f"Aucun fichier XML trouvé dans le ZIP pour {model_id}")
                return None

            xml_filename = xml_files[0]  # Prendre le premier fichier .xml
            zip_ref.extract(xml_filename, directory)

            extracted_path = os.path.join(directory, os.path.basename(xml_filename))
            print(f"Fichier SBML extrait : {extracted_path}")

        # Supprimer le fichier ZIP
        os.remove(zip_path)

        return extracted_path

    except Exception as e:
        print(f"Erreur lors du téléchargement du modèle {model_id}: {e}")
        return None

In [14]:
def download_model_with_metadata(model_data, base_directory):
    """
    Télécharge le modèle et ses métadonnées, puis les enregistre dans un fichier zip.
    """
    try:
        model_id = model_data['id']
        #sbml_url = f"https://www.ebi.ac.uk/biomodels/download?mid={model_id}&format=sbml"
        title = model_data.get('name', "").lower()
        keywords = model_data.get('submitter_keywords', "").lower()

        #if not sbml_url:
        #    print(f"Aucune URL trouvée pour le modèle {model_id}.")
        #    return

        # Déterminer le dossier de destination
        if "therapy" in title:
            directory = os.path.join(base_directory, "Therapy")
        elif "immune system" in title:
            directory = os.path.join(base_directory, "Immune_system")
        elif "immune response" in title:
            directory = os.path.join(base_directory, "Immune_response")
        elif "cell" in title:
            directory = os.path.join(base_directory, "Cell_Immune")
        elif "apopto" in title:
            directory = os.path.join(base_directory, "Cell_Immune")
        elif "lympho" in title:
            directory = os.path.join(base_directory, "Cell_Immune")
        elif "regulation" in title:
            directory = os.path.join(base_directory, "Immune_Regulation")
        elif "interaction" in title:
            directory = os.path.join(base_directory, "Immuno_interaction")
        elif "infection" in title:
            directory = os.path.join(base_directory, "Infection")
        elif "memory" in title:
            directory = os.path.join(base_directory, "Memory_Response")
        elif "body" in title:
            directory = os.path.join(base_directory, "Antibody")
        elif "bodies" in title:
            directory = os.path.join(base_directory, "Antibody")
        elif "complement system" in title:
            directory = os.path.join(base_directory, "Complement_System")
        elif "complement_system" in title:
            directory = os.path.join(base_directory, "Complement_System")
        elif "amyloid-beta" in title:
            directory = os.path.join(base_directory, "Nervous_System")
        elif "aβ" in title:
            directory = os.path.join(base_directory, "Nervous_System")
        elif "signal transduction" in title:
            directory = os.path.join(base_directory, "Immune_Signal_Transduction")
        elif "beta arrestin" in title:
            directory = os.path.join(base_directory, "Immune_Signal_Transduction")
        else:
            directory = os.path.join(base_directory, "Immune")
        # Créer le dossier s'il n'existe pas
        os.makedirs(directory, exist_ok=True)

        # Télécharger le fichier SBML
        model_path = download_model_file(model_id, directory)
        if model_path is None:
            return

        # Récupérer les métadonnées complètes
        try:
            full_metadata = s.get_model(model_id)
        except Exception as e:
            print(f"Erreur lors de la récupération des métadonnées complètes pour {model_id}: {e}")
            return

        # Sauvegarder les métadonnées en JSON
        metadata_filename = f"{model_id}_metadata.json"
        metadata_path = os.path.join(directory, metadata_filename)
        with open(metadata_path, 'w', encoding='utf-8') as f:
            json.dump(full_metadata, f, ensure_ascii=False, indent=4)

        # Créer un fichier zip contenant le modèle et ses métadonnées
        zip_filename = os.path.join(directory, f"{model_id}.zip")
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            zipf.write(model_path, os.path.basename(model_path))
            zipf.write(metadata_path, os.path.basename(metadata_path))

        # Supprimer les fichiers temporaires après zippage
        os.remove(model_path)
        os.remove(metadata_path)

        print(f"Modèle {model_id} et ses métadonnées enregistrés dans {zip_filename}")

    except Exception as e:
        print(f"Erreur lors du traitement du modèle {model_data['id']} : {e}")

In [12]:
def main():
    # Requête mise à jour
    query = (
        'immun* AND curationstatus:"Manually curated" AND modelformat:"SBML" AND TAXONOMY:9606 AND NOT submitter_keywords:"Immuno-oncology"'
    )

    # Répertoire principal
    base_directory = "downloaded_models_SBML"
    os.makedirs(base_directory, exist_ok=True)

    # Obtenir tous les modèles
    models = get_all_models(query)

    # Télécharger chaque modèle et les classer
    for model_data in models:
        download_model_with_metadata(model_data, base_directory)

In [15]:
if __name__ == "__main__":
    main()

Page 1 téléchargée, 10 modèles récupérés.
Page 2 téléchargée, 10 modèles récupérés.
Page 3 téléchargée, 10 modèles récupérés.
Page 4 téléchargée, 10 modèles récupérés.
Page 5 téléchargée, 10 modèles récupérés.
Page 6 téléchargée, 10 modèles récupérés.
Page 7 téléchargée, 3 modèles récupérés.


INFO    [bioservices.BioModels:171]:  Saving file BIOMD0000000573.zip


Fichier SBML extrait : downloaded_models_logical\Immune_response\BIOMD0000000573_url.xml
Modèle BIOMD0000000573 et ses métadonnées enregistrés dans downloaded_models_logical\Immune_response\BIOMD0000000573.zip


INFO    [bioservices.BioModels:171]:  Saving file BIOMD0000000684.zip


Fichier SBML extrait : downloaded_models_logical\Memory_Response\BIOMD0000000684_url.xml
Modèle BIOMD0000000684 et ses métadonnées enregistrés dans downloaded_models_logical\Memory_Response\BIOMD0000000684.zip


...

Fichier SBML extrait : downloaded_models_logical\Cell_Immune\Coulibaly2019.xml
Modèle BIOMD0000000867 et ses métadonnées enregistrés dans downloaded_models_logical\Cell_Immune\BIOMD0000000867.zip


INFO    [bioservices.BioModels:171]:  Saving file BIOMD0000001057.zip


Fichier SBML extrait : downloaded_models_logical\Immune\Nikolov2020.xml
Modèle BIOMD0000001057 et ses métadonnées enregistrés dans downloaded_models_logical\Immune\BIOMD0000001057.zip


INFO    [bioservices.BioModels:171]:  Saving file BIOMD0000000599.zip


Fichier SBML extrait : downloaded_models_logical\Immune_Signal_Transduction\BIOMD0000000599_url.xml
Modèle BIOMD0000000599 et ses métadonnées enregistrés dans downloaded_models_logical\Immune_Signal_Transduction\BIOMD0000000599.zip
